<a href="https://colab.research.google.com/github/arav38/LangChain-Framework---Basic-AI-Agent/blob/main/Langgraph_Framework_AI_Agent_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langgraph langchain_community langchain_community langchain langchain-groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.5/152.5 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 14.2 MB/s eta 0:00:00


In [2]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.graph import START,END
from langchain import chat_models, LLMChain, PromptTemplate
from langchain.schema import BaseOutputParser
from langchain_groq import ChatGroq

In [3]:
llm = ChatGroq(model ="meta-llama/llama-4-scout-17b-16e-instruct",api_key='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')

In [4]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7c56d5ca2850>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7c56d58d7a90>, model_name='meta-llama/llama-4-scout-17b-16e-instruct', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
# get data from wikipedia

In [6]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=3e0ee1630c5578ae0f8a4b75ae20a2163ad751beb7f7b7ec0144bf76177ad663
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


In [7]:
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper

In [8]:
wikipedia_tool =  WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

In [9]:
from langchain.tools import Tool

In [10]:
wikipedia_agent =  Tool.from_function(
    name ='wikipendi_agent',
    func=wikipedia_tool.run,
    description='useful for when you need to answer question about current affairs '

)

In [ ]:
# from langchain.utilities import OpenWeatherMapAPIWrapper
# from langchain.tools import

In [ ]:
wikipedia_agent.run("Whos Donald Trump")

'Page: Trump family\nSummary: The Trump family is the prominent wealthy family of US President Donald Trump. The family is of Bavarian German and Scottish descent. They are active in business, entertainment, politics, and real estate. Prominent members such as the President\'s grandfather Friedrich Trump and his father Fred Trump are grouped here.\n\nPage: Donald Trump Jr.\nSummary: Donald John Trump Jr. (born December 31, 1977), often nicknamed Don Jr., is an American businessman and political activist. He is the eldest child of U.S. president Donald Trump and his first wife Ivana.\nTrump serves as a trustee and executive vice president of the Trump Organization, running the company alongside his younger brother Eric. During their father\'s first presidency, the brothers continued to engage in deals and investments in foreign countries and collect payments at their U.S. properties from foreign governments, despite pledging not to do so. He also served as a boardroom judge on the reali

In [11]:
from langchain.prompts import PromptTemplate

In [12]:
intent_prompt = PromptTemplate.from_template("""
Classify the following user input into one of the categories: insurance, finance, wikipedia, fallback.

User input: {input}
Category:
""")

In [13]:
def route_intent(input_text):
  input_prompt = intent_prompt.format(input_text)
  responce =llm.predict(input_prompt).strip().lower()
  return responce

In [14]:
from langgraph.graph import START,END
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from pydantic import BaseModel,Field
from typing import Literal

In [15]:
from pydantic import BaseModel, Field
from typing import Literal

class MessageClassifier(BaseModel):
    message_type: Literal["Insurance", "Medical", "Financial", "Wikepedia"] = Field(
        ...,
        description="Classify message whether it is insurance, medical, financial or wikipedia"
    )


In [16]:
class State(TypedDict):
  message : Annotated[list,add_messages]
  message_type : str | None

In [17]:
def router(state: State):
    message_type = state.get("message_type", "Wikipedia")
    if message_type == "Financial":
        return {"next": "Financial"}
    elif message_type == "Insurance":
        return {"next": "Insurance"}
    elif message_type == "Medical":
        return {"next": "Medical"}

    return {"next": "Wikipedia"}

In [22]:
def message_classify(state:State):

  last_message = state["message"][-1]
  classifier_llm = llm.with_structured_output(MessageClassifier)

  result = llm.invoke([
      {"role":"system","context":"""Classify user message either Insurance Related, Medical Related , Finanacial Related or Current Affair related
           Insurance Related - If it ask about insurance then please answer question with maximum accuracy based on data
           Finanacial Related - if it asked about finance then please answer question with maximum accuracy based on data
           Medical Related - if it asked about medical then please answer question based on data with maximum accuracy
           Wikipendia - Please classify question based
             """
           },
        {"role":"user","context":last_message.content}
])

  return {"message_type":result.message_type}




def classify_message(state:State):

  return {"message_type":result.content}



In [23]:
from decimal import Context
def insurace_bot():

  last_message = State["message"][-1]

  message = [
      {"role":"system",
      "context": """ You are an intelligent and helpful insurance assistant. You help users understand and manage their insurance policies, claims, and benefits. Always respond clearly, professionally, and accurately.
      """},
      {
      "role":"user",
      "context" : last_message.content}
  ]

  reply = llm.invoke(message)

  return {"Message":[{"role":"user","Context":reply.content}]}

In [24]:
def medical_bot():

  last_message = State["message"][-1]

  message = [
      {"role":"system",
      "context": """ You are an intelligent and helpful Medical assistant. You help users understand and manage their insurance policies, claims, and benefits. Always respond clearly, professionally, and accurately.
      """},
      {
      "role":"user",
      "context" : last_message.content}
  ]

  reply = llm.invoke(message)

  return {"Message":[{"role":"user","Context":reply.content}]}

In [25]:
def financial_bot():


  last_message = State["message"][-1]

  message = [
      {"role":"system",
      "context": """ You are an intelligent and helpful Financial assistant. You help users understand and manage their insurance policies, claims, and benefits. Always respond clearly, professionally, and accurately.
      """},
      {
      "role":"user",
      "context" : last_message.content}
  ]

  reply = llm.invoke(message)

  return {"Message":[{"role":"user","Context":reply.content}]}


In [26]:
from os import stat
graph_builder = StateGraph(State)

In [27]:
graph_builder.add_node("Classifier",MessageClassifier)


In [28]:
graph_builder.add_node("router",router)
graph_builder.add_node("Wikiepedia",wikipedia_agent)
graph_builder.add_node("Insurance",insurace_bot)
graph_builder.add_node("Medical",medical_bot)
graph_builder.add_node("Financial",financial_bot)


In [29]:
graph_builder.add_edge(START,"Classifier")
graph_builder.add_edge("Classifier","router")

In [30]:
graph_builder.add_conditional_edges('router',lambda state:state.get("Next"),
                                   { "Insurance":"Insurance","Medical":"Medical","Financial":"Financial","Wikiepedia":"Wikiepedia", None: END})

In [31]:
graph_builder.add_edge("Wikiepedia",END)
graph_builder.add_edge("Insurance",END)
graph_builder.add_edge("Medical",END)
graph_builder.add_edge("Financial",END)


In [32]:
graph = graph_builder.compile()

In [ ]:
def run_chatbot():
    # Initialize state according to AgentState definition
    state = {"messages": []} # Removed message_type

    while True:
        user_input = input("Message: ")
        if user_input == "exit":
            print("Bye")
            break

        # Ensure updates conform to AgentState
        state["messages"] = state.get("messages", []) + [
            {"role": "user", "content": user_input}
        ]

        # The 'next' key will be managed by the graph's conditional edges

        try:
            state = graph.invoke(state)
        except Exception as e:
            print(f"An error occurred during graph invocation: {e}")
            # Optionally, print the state to help with debugging
            # print(f"Current state before error: {state}")
            continue # Continue the loop to allow more input or exit

        if state.get("messages") and len(state["messages"]) > 0:
            last_message = state["messages"][-1]
            # Check if last_message is a dictionary with 'content' key
            if isinstance(last_message, dict) and 'content' in last_message:
                 print(f"Assistant: {last_message['content']}")
            else:
                 # If the message format is unexpected, print the whole message object
                 print(f"Assistant (raw message): {last_message}")


if __name__ == "__main__":
    run_chatbot()

Message: hello
An error occurred during graph invocation: BaseModel.__init__() takes 1 positional argument but 2 were given


In [ ]:
from typing import List, Tuple, Annotated, TypedDict

from langgraph.graph.message import AnyMessage
from langgraph.graph import add_message
from langgraph.graph.state import StateGraph

class AgentState(TypedDict):
    # The list of messages exchanged between the user and the chatbot.
    messages: Annotated[List[AnyMessage], add_message]
    # The next node to visit
    next: str

ImportError: cannot import name 'add_message' from 'langgraph.graph' (/usr/local/lib/python3.11/dist-packages/langgraph/graph/__init__.py)

In [ ]:
# Assuming graph_builder is already defined, re-initialize it with AgentState
# If graph_builder is not defined yet, use this to initialize it
graph_builder = StateGraph(AgentState)

# Now proceed with adding nodes and edges using this graph_builder
# e.g., graph_builder.add_node("router", router)
#       graph_builder.add_edge(START, "Classifier")
# ... and so on with the rest of your graph definition